In [1]:
from keras.models import load_model
import cv2
import numpy as np
from random import choice

In [2]:
REV_CLASS_MAP = {
    0: "paper",
    1: "Stone",
    2: "scissors",
    
}


def mapper(val):
    return REV_CLASS_MAP[val]


In [3]:
def calculate_winner(move1, move2):
    if move1 == move2:
        return "Tie"

    if move1 == "Stone":
        if move2 == "scissors":
            return "User"
        if move2 == "paper":
            return "Computer"

    if move1 == "paper":
        if move2 == "Stone":
            return "User"
        if move2 == "scissors":
            return "Computer"

    if move1 == "scissors":
        if move2 == "paper":
            return "User"
        if move2 == "Stone":
            return "Computer"

In [5]:
cap = cv2.VideoCapture(0)
import urllib

prev_move = None

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # rectangle for user to play
    cv2.rectangle(frame, (50, 100), (350, 400), (0, 255, 255), 2)
    # rectangle for computer to play
    #cv2.rectangle(frame, (600, 100), (1000, 300), (0, 255, 255), 2)

    # extract the region of image within the user rectangle
    roi = frame[50:350, 100:400]
    img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (150, 150))
    
    model = load_model("final.h5")

    # predict the move made
    pred = model.predict(np.array([img]))
    move_code = np.argmax(pred[0])
    user_move_name = mapper(move_code)

    # predict the winner (human vs computer)
    if prev_move != user_move_name:
        if user_move_name != "none":
            computer_move_name = choice(['Stone', 'paper', 'scissors'])
            winner = calculate_winner(user_move_name, computer_move_name)
        else:
            computer_move_name = "none"
            winner = "Waiting..."
    prev_move = user_move_name

    # display the information
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, "Your Move: " + user_move_name,
                (50, 50), font, 1, (255, 0, 255), 2)
    cv2.putText(frame, "Computer's Move: " + computer_move_name,
                (50, 450), font, 1, (255, 0, 255), 2)
    cv2.putText(frame, "Winner: " + winner,
                (360, 250), font, 1, (0, 0, 255), 2)

    
    cv2.imshow("Stone Paper Scissors", frame)
    cv2.resizeWindow('image', 2000,2000)

    k = cv2.waitKey(10)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()